# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf

In [19]:
df = pd.read_csv('census_data.csv')
df.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

### Perform a Train Test Split on the Data

In [20]:
# create a label
# NB:  ' >50K' not '>50K'
df['label'] = 0
df.loc[df['income_bracket']==" >50K",'label'] = 1

# split the data
y_val = df['label']
x_data = df.drop('income_bracket',axis=1).drop('label',axis=1)

test_size, seedval = .30, 101

# I don't like his notation but I am adopting.  I would normally have X_test or X_train
# as df_test & df_train
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,
                                        test_size=test_size,random_state=seedval)



In [21]:
x_data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [22]:
# cats
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

# numerics
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [23]:
input_func_train=tf.estimator.inputs.pandas_input_fn(
    x=X_train,y=y_train,batch_size=250,
    num_epochs=None,shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [24]:
LCM_model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': 1, '_model_dir': '/var/folders/9y/n0rxh9rj60q4v8zgsclp0gm97tm33p/T/tmp3gnxeb2c', '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5}


** Train your model on the data, for at least 5000 steps. **

In [25]:
LCM_model.train(input_fn=input_func_train,steps=10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/9y/n0rxh9rj60q4v8zgsclp0gm97tm33p/T/tmp3gnxeb2c/model.ckpt.
INFO:tensorflow:loss = 173.287, step = 1
INFO:tensorflow:global_step/sec: 172.837
INFO:tensorflow:loss = 820.552, step = 101 (0.580 sec)
INFO:tensorflow:global_step/sec: 187.308
INFO:tensorflow:loss = 439.485, step = 201 (0.534 sec)
INFO:tensorflow:global_step/sec: 235.911
INFO:tensorflow:loss = 134.331, step = 301 (0.424 sec)
INFO:tensorflow:global_step/sec: 252.953
INFO:tensorflow:loss = 135.146, step = 401 (0.395 sec)
INFO:tensorflow:global_step/sec: 269.157
INFO:tensorflow:loss = 264.662, step = 501 (0.372 sec)
INFO:tensorflow:global_step/sec: 253.213
INFO:tensorflow:loss = 651.25, step = 601 (0.395 sec)
INFO:tensorflow:global_step/sec: 230.185
INFO:tensorflow:loss = 102.806, step = 701 (0.435 sec)
INFO:tensorflow:global_step/sec: 186.469
INFO:tensorflow:loss = 138.464, step = 801 (0.536 sec)
INFO:tensorflow:global_step/s

INFO:tensorflow:global_step/sec: 236.85
INFO:tensorflow:loss = 328.908, step = 8401 (0.422 sec)
INFO:tensorflow:global_step/sec: 236.555
INFO:tensorflow:loss = 83.9438, step = 8501 (0.423 sec)
INFO:tensorflow:global_step/sec: 180.52
INFO:tensorflow:loss = 299.983, step = 8601 (0.554 sec)
INFO:tensorflow:global_step/sec: 244.207
INFO:tensorflow:loss = 315.083, step = 8701 (0.409 sec)
INFO:tensorflow:global_step/sec: 236.924
INFO:tensorflow:loss = 151.669, step = 8801 (0.422 sec)
INFO:tensorflow:global_step/sec: 197.342
INFO:tensorflow:loss = 79.0574, step = 8901 (0.506 sec)
INFO:tensorflow:global_step/sec: 236.592
INFO:tensorflow:loss = 175.139, step = 9001 (0.423 sec)
INFO:tensorflow:global_step/sec: 238.578
INFO:tensorflow:loss = 136.051, step = 9101 (0.419 sec)
INFO:tensorflow:global_step/sec: 246.512
INFO:tensorflow:loss = 87.0456, step = 9201 (0.406 sec)
INFO:tensorflow:global_step/sec: 175.25
INFO:tensorflow:loss = 205.826, step = 9301 (0.571 sec)
INFO:tensorflow:global_step/sec: 

### Evaluation


** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [26]:
input_func_test = tf.estimator.inputs.pandas_input_fn(
    x=X_test,
    batch_size=len(X_test),
    shuffle=False)

In [27]:
predictions = list( LCM_model.predict(input_func_test) )

INFO:tensorflow:Restoring parameters from /var/folders/9y/n0rxh9rj60q4v8zgsclp0gm97tm33p/T/tmp3gnxeb2c/model.ckpt-10000


** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [28]:
results = []
for prediction in predictions:
    results.append( prediction['class_ids'][0] )


In [29]:
results[0:25]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1]

In [17]:
len(y_val)

32561

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [30]:
# use notation of the function as documented.
y_true, y_pred = y_test, results
cr = classification_report(y_true, y_pred)

In [31]:
print(cr)

             precision    recall  f1-score   support

          0       0.89      0.90      0.90      7436
          1       0.67      0.65      0.66      2333

avg / total       0.84      0.84      0.84      9769



# Great Job!